In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ROOT
import os
ROOT.enableJSVis()
c1 = ROOT.TCanvas()
ROOT.gSystem.Load("/usr/local/software/quadis/build/lib/libquadis.so")   

Welcome to JupyROOT 6.10/00


0

In [2]:
data_ind = []
num_files=0
f = open('rt_data_ind', 'r')
while True:
    line = f.readline()
    if not line:
        break
    else:
        temp = line.strip("\n")
        data_ind.append(int(temp))
    num_files = num_files+1
f.close()

baseline1 = np.zeros(num_files)
infile = ROOT.TFile("/home/vasu/PPC/PPC_SigGen/SCAN100004/SCAN100004_RUN13.root")
tree = infile.Get("T1")
nevent = ROOT.NEvent()
tree.SetBranchAddress("NEvent",nevent)
fitter = ROOT.NAPulseFitter("/home/vasu/PPC/PPC_SigGen/SignalLib_root/new_OutputRZperfect.root","T1","NEvent",1)
baselines = 1000
taus = 7100
rises=750
gaps=225
for i in range (num_files):
    j = data_ind[i]
    tree.GetEntry(j)
    npulse = nevent.GetPulse(0)
    sP = npulse.GetSamplingPeriod()
    pulse = fitter.GetPreProcessedPulse(npulse, False, baselines, taus, rises, gaps)
    blrPulse = ROOT.NPulse()
    blrFilter = ROOT.NFBaselineRemover(1000*sP) #use 1000 samples to calculate the baseline
    blrFilter.Filter(pulse,blrPulse)# apply the filter
    baseline = blrFilter.GetCalcParameterByName("Baseline")
    trapPulse = ROOT.NPulse() #to hold the filtered result
    trapFilter = ROOT.NFTrapezoid(20*sP,10*sP,6750,baseline) #tau of about 6000 samples
    trapFilter.SetBaseline(baseline)
    trapFilter.Filter(pulse,trapPulse)
    npcFilter = ROOT.NPCMinMax()
    npcFilter.CalculateParameters(trapPulse)
    blrPC = ROOT.NPCBaseline(2000*sP) 
    alist = blrPC.GetParameterNames() #get all of the available parameters
    blrPC.CalculateParameters(trapPulse) #calculate all of the parameters
    baseline1[i] = blrPC.GetCalcParameterByName("Baseline")
    if(baseline1[i]>0):
        continue
    else:
        print j
        print baseline1[i]

15
-0.00037145268683387334
37
-0.00018511858576142123
86
-8.387552938005714e-05
100
-0.00014572759106765455
154
-0.00011574254467160397
179
-6.566703163606873e-05
250
-0.0002082254248272473
357
-7.968235345847591e-05
457
-8.596636728664684e-05
474
-0.00011209729316037889
477
-0.0002209364255813076
533
-0.00010928122949548209
591
-6.417436478707358e-05
596
-9.998023722055975e-05
598
-5.8560345054859545e-05
626
-0.00044957024803860836
644
-4.803297229583368e-06
716
-9.817987412857985e-05
803
-9.081934399655734e-05
832
-4.2175578542590444e-05
885
-0.00030517353717288387
908
-8.850587616767684e-05
922
-0.00013304116156511537
1002
-0.00016358064195087747
1033
-0.0001578074413900355
1046
-0.00020518923673755953
1069
-8.651272642463319e-05
1088
-0.0002057810362951254
1092
-0.0009766905696954166
1124
-0.00012398654074472388
1147
-3.019428026168946e-05
1201
-0.000294840955387569
1202
-7.248220987078152e-05
1210
-0.00017446016161496952
1216
-8.599773304917432e-05
1316
-2.9948904919206583e-05
134

In [3]:
hbaseline = ROOT.TH1F("hbaseline","hbaseline", 1000,-0.5,0.5)
for i in range (num_files):
    hbaseline.Fill(baseline1[i])
hbaseline.Draw()
c1.Draw()